In [1]:
import gzip
import json
import ipdb
from pprint import pprint
from celan_app.models import Verse, Word, WordTranslation

In [25]:
def check_trans(data):
    words = Word.objects.all()
    is_trans = []
    no_trans = []
    for word in words:
        if word.lemma in wiki_dict:
            is_trans.append(word.lemma)
        else:
            no_trans.append(word.lemma)
    print(f"number of words with no translations = {len(no_trans)} ")
    print(f"number of words with translations = {len(is_trans)} ")
    return no_trans, is_trans

In [4]:
def read_file(path):
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    print(f"length of file = {len(data)}")
    return data

In [3]:
def save_file(path, file_to_save):
    with open(path, "w", encoding="utf-8") as f:
        f.write(json.dumps(file_to_save))


In [32]:
path_wiki_dict = "data/wiki_dict.json"
wiki_dict = read_file(path_wiki_dict)

length of file = 98419


In [33]:
list(wiki_dict.items())[100]

('Familie',
 [{'sense': '', 'eng': 'family'},
  {'sense': '', 'rus': 'семья'},
  {'sense': '', 'rus': 'семейство'},
  {'sense': '', 'ukr': 'родина'},
  {'sense': '', 'ukr': 'сімейство'}])

In [ ]:
# previous results:
# number of words with no translations = 4849 
# number of words with translations = 3540 
no_trans, is_trans = check_trans(wiki_dict)

In [9]:
path = "data/no_trans_2.json"
# save_file(path, no_trans)

In [19]:
def find_word(path, word):
    progress_check = range(0, 10_000_000, 100_000)
    with gzip.open(path, "rt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            unpacked_line = json.loads(line)
            try:
                if i in progress_check:
                    print(i)
                if unpacked_line["word"] == word:
                    return unpacked_line
            except:
                pass

In [ ]:
path = "data/de_wiki.gz"
# item = find_word(path, "Sandkunst")

In [6]:
list(wiki_dict.items())[100]

('Familie',
 [{'sense': '', 'eng': 'family'},
  {'sense': '', 'rus': 'семья'},
  {'sense': '', 'rus': 'семейство'},
  {'sense': '', 'ukr': 'родина'},
  {'sense': '', 'ukr': 'сімейство'}])

In [61]:
def populate_word_translation(wiki_dict):
    progress_check = range(0, 10_000_000, 3_000)
    languages = {"eng": "en", "ukr": "uk", "rus": "ru"}
    words = Word.objects.all()
    count = 0
    for i, word_obj in enumerate(words):
        lemma = word_obj.lemma
        values = wiki_dict.get(lemma, None)
        if not values:
            continue
        for value in values:
            # {'sense': '', 'eng': 'family'}
            sense = value["sense"]
            lng = list(value.keys())[1]
            lang = languages[lng]
            trans = value[lng]
            # all is_prefered = False as in default
            WordTranslation.objects.create(
                word = word_obj,
                lang = lang,
                sense = sense,
                trans = trans,
            )
            if i in progress_check:
                print(i)
            count += 1
    return f"number of WordTranslation instances = {WordTranslation.objects.count()}" 
        

In [62]:
# populate_word_translation(wiki_dict)

0
0
0


'number of WordTranslation instances = 18408'

In [54]:
# WordTranslation.objects.all().delete()

(18408, {'celan_app.WordTranslation': 18408})

In [68]:
zeit = Word.objects.get(lemma = "zu")

In [69]:
list(zeit.word_translations.values_list("sense", "lang", "trans"))

[('in Richtung auf', 'en', 'to'),
 ('in Richtung auf', 'ru', 'к'),
 ('in Richtung auf', 'uk', 'до'),
 ('in Bezug auf', 'en', 'at'),
 ('in Bezug auf', 'ru', 'на'),
 ('zu der Zeit als, zu der Zeit innerhalb eines Zeitraumes', 'ru', 'в'),
 ('zu der Zeit als, zu der Zeit innerhalb eines Zeitraumes', 'uk', 'на'),
 ('zu der Zeit als, zu der Zeit innerhalb eines Zeitraumes', 'uk', 'в'),
 ('umgangssprachlich: geschlossen', 'en', 'closed'),
 ('umgangssprachlich: geschlossen', 'ru', 'закрыто'),
 ('umgangssprachlich: geschlossen', 'uk', 'зачинений'),
 ('drückt vor einem Adjektiv aus, dass sein Grad höher ist als gewünscht',
  'en',
  'too'),
 ('drückt vor einem Adjektiv aus, dass sein Grad höher ist als gewünscht',
  'ru',
  'слишком'),
 ('umgangssprachlich: geschlossen', 'en', 'closed')]

# Add translations missing in wiki

In [5]:
path_no_trans_with_trans = "data/no_trans_with_trans.json"
no_trans_with_trans = read_file(path_no_trans_with_trans)

length of file = 4157


In [6]:
pprint(list(no_trans_with_trans.items())[0])

('Augenglut',
 [{'eng': 'eyes-glut',
   'sense': 'Kontext (aus „Entschlackt“, Vers #13709): zt Messer wären, '
            'blankgezogen wie damals im Laubengang zu Paris, eine Augenglut '
            'lang, der arktische Stier käme gesprungen und bekrönte mit uns '
            'seine'},
  {'rus': '',
   'sense': 'Kontext (aus „Entschlackt“, Vers #13709): zt Messer wären, '
            'blankgezogen wie damals im Laubengang zu Paris, eine Augenglut '
            'lang, der arktische Stier käme gesprungen und bekrönte mit uns '
            'seine'},
  {'sense': 'Kontext (aus „Entschlackt“, Vers #13709): zt Messer wären, '
            'blankgezogen wie damals im Laubengang zu Paris, eine Augenglut '
            'lang, der arktische Stier käme gesprungen und bekrönte mit uns '
            'seine',
   'ukr': ''}])


In [83]:
# populate_word_translation(no_trans_with_trans)

3000
3000
3000
6000
6000
6000


'number of WordTranslation instances = 34815'

In [89]:
list(Word.objects.get(lemma = "Laub").word_translations.values_list())

[(24323,
  34718,
  'en',
  'leaves',
  'Botanik: Gesamtheit der Blattorgane von Laubbäumen und Sträuchern',
  False,
  datetime.datetime(2026, 1, 7, 8, 28, 49, 378712, tzinfo=datetime.timezone.utc)),
 (24324,
  34718,
  'en',
  'foliage',
  'Botanik: Gesamtheit der Blattorgane von Laubbäumen und Sträuchern',
  False,
  datetime.datetime(2026, 1, 7, 8, 28, 49, 379430, tzinfo=datetime.timezone.utc)),
 (24325,
  34718,
  'ru',
  'листва',
  'Botanik: Gesamtheit der Blattorgane von Laubbäumen und Sträuchern',
  False,
  datetime.datetime(2026, 1, 7, 8, 28, 49, 380138, tzinfo=datetime.timezone.utc)),
 (24326,
  34718,
  'en',
  'spades',
  'Farbe beim deutschen Kartenspiel mit einem Blatt (häufig Lindenblatt) als Symbol',
  False,
  datetime.datetime(2026, 1, 7, 8, 28, 49, 380868, tzinfo=datetime.timezone.utc))]

# Mark neologisms

In [15]:
def mark_neologisms(data):
    languages = {"eng": "en", "ukr": "uk", "rus": "ru"}
    progress_check = range(0, 10_000_000, 3_000)
    count = 0
    for i, (word, values) in enumerate(data.items()):
        trans_objs = WordTranslation.objects.filter(word__lemma = word)
        for trans_obj in trans_objs:
            trans_obj.is_neologizm = True
            trans_obj.save()
        if i in progress_check:
            print(i)
        count += 1
    return f"number of WordTranslation instances = {WordTranslation.objects.count()}" 

In [16]:
mark_neologisms(no_trans_with_trans)

0
3000


'number of WordTranslation instances = 34815'

In [23]:
list(Word.objects.get(lemma = "Laub").word_translations.all())[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x71397b8df170>,
 'id': 24323,
 'word_id': 34718,
 'lang': 'en',
 'trans': 'leaves',
 'sense': 'Botanik: Gesamtheit der Blattorgane von Laubbäumen und Sträuchern',
 'is_neologism': False,
 'is_preferred': False,
 'created_at': datetime.datetime(2026, 1, 7, 8, 28, 49, 378712, tzinfo=datetime.timezone.utc)}

<WordTranslation: leaves>